In [ ]:
import arcpy
from arcpy import env
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor
from pykrige.ok import OrdinaryKriging

In [ ]:
# 设置工作环境
work_path = "D:\ArcGISProjects\syraster\ph.gdb"
env.workspace = work_path

In [ ]:
# 读取DEM数据并转换为NumPy数组
dem_raster = "SY_DEM_5" # 高程
asp_raster = "SY_ASP_5" # 坡向
slp_raster = "SY_SL_5" # 坡度
pre_raster = "pre_2022" # 降水
tmp_raster = "tmp_2022" # 气温
ndvi_raster = "ndvi_2022" # NDVI
twi5_raster = "SY_TWI_5" # TWI
tpi5_raster = "SY_TPI_3" # TPI
tpi11_raster = "SY_TPI_11" # TPI
tpi101_raster = "SY_TPI_101" # TPI
tpi201_raster = "SY_TPI_201" # TPI


In [5]:
import sklearn.externals
import joblib
import pandas as pd
import numpy as np

In [6]:
# 读取数据
# 读取数据
data = pd.read_csv('../data/csv_file/test_data.csv')

In [7]:
# 删除缺失值
# 删除有缺失值的行
data.dropna(inplace=True)
missing_values = data.isnull().sum()
missing_values

Unnamed: 0    0
X             0
Y             0
pre           0
tmp           0
asp           0
slope         0
dem           0
ndvi          0
twi5          0
tpi3          0
tpi11         0
tpi101        0
tpi201        0
dl            0
ph            0
dtype: int64

In [8]:
data.drop(['Unnamed: 0'], axis=1, inplace=True)
data.columns

Index(['X', 'Y', 'pre', 'tmp', 'asp', 'slope', 'dem', 'ndvi', 'twi5', 'tpi3',
       'tpi11', 'tpi101', 'tpi201', 'dl', 'ph'],
      dtype='object')

In [9]:
data.drop(columns=['dl'],inplace=True)

In [12]:
# 划分测试数据
train_data = data.sample(frac=0.7,random_state=0)   # 80%的数据用于训练
test_data = data.drop(train_data.index)                 # 20%的数据用于测试
train_data.sample()
label = 'ph'